In [63]:
import os
import re
import gc
import glob
import pickle
import warnings
import os.path
import itertools
import numpy as np
import pandas as pd
import pandas.io.common

from tqdm import tqdm
from datetime import datetime
warnings.filterwarnings("ignore")

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

## READING Data Sources Features

In [64]:
## Anomaly Data (Baseline data)
loc_path = r'/Users/raj2.gaurav/Desktop/' 
anomaly_features_path = loc_path + 'Causal Model/04. Feature Store/01. Anomaly Features/'
anomaly_df = pd.read_parquet(anomaly_features_path + 'Anomaly_features.parquet')
anomaly_df_br=anomaly_df[anomaly_df['Circle']=='bihar']

In [65]:
## LSR Feature 
lsr_features_path = loc_path + 'Causal Model/04. Feature Store/02. LSR Features/'
lsr_df = pd.read_parquet(lsr_features_path + 'Lsr_features.parquet')

In [66]:
## Network Health Feature
ccl_features_path = loc_path + 'Causal Model/04. Feature Store/03. Nework Health Features/'
ccl_df = pd.read_parquet(ccl_features_path + 'Netowork_features.parquet')

In [18]:
## PRB Feature
prb_features_path = loc_path + 'Causal Model/04. Feature Store/04. PRB Features/'
prb_df = pd.read_parquet(prb_features_path + 'Prb_features.parquet')

In [67]:
anomaly_df_br.head()

,Circle,City,Component,Partition_date,Hour,Total customers,Anomaly,Ratio data,Ratio voice,Ratio hsi,Cluster_issue,Data volume,Data volume formula,Data volume mean,Data volume std,Voice affected users,Voice affected users formula,Voice affected users mean,Voice affected users std,Hsi affected users,Hsi affected users formula,Hsi affected users mean,Hsi affected users std,Num_grids,Num_cells,Num_sectors,Indoor frqeqency % 850,Indoor frqeqency % 1800,Indoor frqeqency % 2300,Prb_10,Prb_10_25,Prb_25_70,Prb_70_90,Prb_90,Target_Cluster
25005,bihar,deoghar,33,2022-11-06,5,4726,Voice,1,4,2,None,27,49,16,10,267,223,62,53,65,223,25,66,9,35,10,31,31,36,9.00,22.00,4.00,0.00,0.00,0
25006,bihar,deoghar,33,2022-11-06,6,6551,Voice,1,3,2,None,47,85,27,19,499,495,144,116,160,447,73,124,9,35,10,35,31,33,6.00,19.00,10.00,0.00,0.00,0
25007,bihar,katihar,116,2022-11-06,11,13276,HSI,2,2,4,high_prb,137,191,46,48,888,1478,352,375,1533,1532,337,398,8,27,8,32,39,27,0.00,0.00,8.00,8.00,11.00,1
25008,bihar,katihar,116,2022-11-06,12,14047,HSI,2,2,4,high_prb,141,197,47,49,851,1454,342,370,1656,1616,351,421,8,27,8,33,40,26,0.00,0.00,7.00,10.00,10.00,1
25009,bihar,patna,85,2022-11-06,7,11028,Data - Voice,4,3,0,None,166,129,35,31,991,887,252,211,113,674,131,180,13,60,19,30,31,38,0.00,11.00,41.00,5.00,3.00,0


In [71]:
anomaly_df_br['Partition_date'].unique()

array(['2022-11-06', '2022-11-07', '2022-11-08', '2022-11-04',
       '2022-11-05', '2022-11-09', '2022-11-03'], dtype=object)

In [70]:
anomaly_df_br['Partition_date'].replace('44873','2022-11-08',inplace=True)

In [72]:
anomaly_df_br.shape

(33310, 35)

In [73]:
lsr_df.head()

,Circle,City,Partition_date,Component,Hour,Sum_total_volume_mb_download,Sum_total_volume_mb_upload,Unique_customers,Total_duration_sec,Total_sessions,Rsrp_nom,Rsrp_dnom,Rsrq_nom,Rsrq_dnom,Mean_cqi_nom,Mean_cqi_dnom,Pusch_sinr_nom,Pusch_sinr_dnom,Ta_distance_nom,Ta_distance_dnom,Intersite_distance_nom,Intersite_distance_dnom,Rsrp_nom_lag_1_day,Rsrp_dnom_lag_1_day,Rsrq_nom_lag_1_day,Rsrq_dnom_lag_1_day
275585,bihar,bagaha,2022-11-03,50,3,979962576.00,41459135.00,301,68040,3016,-154641.00,1525,-16045.00,1525,30920.00,3003,39813.70,2839,3584212.00,3016,6252803.00,3016,-154641.00,1525.00,-16045.00,1525.00
183716,bihar,jamshedpur,2022-11-03,114,21,164019253862.00,18363790274.00,9237,4759403,116366,-8589648.00,93517,-1339488.50,93517,924133.00,115784,771762.40,57458,26652000.00,116366,38538303.00,116366,-8589648.00,93517.00,-1339488.50,93517.00
183717,bihar,bokaro_steel_city,2022-11-03,123,21,59475807199.00,5055020583.00,2882,1465781,26667,-1643720.00,17635,-248222.50,17635,234934.00,26514,236453.10,16151,9583052.00,26667,19986012.00,26667,-1643720.00,17635.00,-248222.50,17635.00
183718,bihar,chirkunda,2022-11-03,44,21,24801019147.00,1401710779.00,1050,620816,10419,-622451.00,5797,-79287.00,5797,89335.00,10341,69489.10,7968,14705718.00,10419,17250691.00,10419,-622451.00,5797.00,-79287.00,5797.00
183719,bihar,lakhisarai,2022-11-03,17,21,61738316591.00,5008062642.00,3670,1586822,36777,-2925510.00,30966,-445793.50,30966,277519.00,36585,264499.60,21778,18127887.00,36777,29238084.00,36777,-2925510.00,30966.00,-445793.50,30966.00


In [74]:
lsr_df.shape

(964551, 26)

In [75]:
anomaly_df_br.dtypes

Circle                           object
City                             object
Component                         int64
Partition_date                   object
Hour                              int64
Total customers                   int64
Anomaly                          object
Ratio data                        int64
Ratio voice                       int64
Ratio hsi                         int64
Cluster_issue                    object
Data volume                       int64
Data volume formula               int64
Data volume mean                  int64
Data volume std                   int64
Voice affected users              int64
Voice affected users formula      int64
Voice affected users mean         int64
Voice affected users std          int64
Hsi affected users                int64
Hsi affected users formula        int64
Hsi affected users mean           int64
Hsi affected users std            int64
Num_grids                         int64
Num_cells                         int64


In [76]:
lsr_df.dtypes

Circle                           object
City                             object
Partition_date                   object
Component                         int64
Hour                              int32
Sum_total_volume_mb_download     object
Sum_total_volume_mb_upload       object
Unique_customers                  int64
Total_duration_sec                int64
Total_sessions                    int64
Rsrp_nom                         object
Rsrp_dnom                         int64
Rsrq_nom                         object
Rsrq_dnom                         int64
Mean_cqi_nom                    float64
Mean_cqi_dnom                     int64
Pusch_sinr_nom                   object
Pusch_sinr_dnom                   int64
Ta_distance_nom                  object
Ta_distance_dnom                  int64
Intersite_distance_nom           object
Intersite_distance_dnom           int64
Rsrp_nom_lag_1_day               object
Rsrp_dnom_lag_1_day             float64
Rsrq_nom_lag_1_day               object


In [77]:
lsr_df['Hour']=lsr_df['Hour'].astype('int64')

In [ ]:
lsr_df[lsr_df['City']=='patna' & lsr_df['City']=='patna']

In [ ]:
## Joining with Lsr Data

In [78]:
anomaly_lsr_df=pd.merge(anomaly_df_br,lsr_df,on=['Circle','City','Partition_date','Component','Hour'],how='left')

In [80]:
anomaly_lsr_df.head()

,Circle,City,Component,Partition_date,Hour,Total customers,Anomaly,Ratio data,Ratio voice,Ratio hsi,Cluster_issue,Data volume,Data volume formula,Data volume mean,Data volume std,Voice affected users,Voice affected users formula,Voice affected users mean,Voice affected users std,Hsi affected users,Hsi affected users formula,Hsi affected users mean,Hsi affected users std,Num_grids,Num_cells,Num_sectors,Indoor frqeqency % 850,Indoor frqeqency % 1800,Indoor frqeqency % 2300,Prb_10,Prb_10_25,Prb_25_70,Prb_70_90,Prb_90,Target_Cluster,Sum_total_volume_mb_download,Sum_total_volume_mb_upload,Unique_customers,Total_duration_sec,Total_sessions,Rsrp_nom,Rsrp_dnom,Rsrq_nom,Rsrq_dnom,Mean_cqi_nom,Mean_cqi_dnom,Pusch_sinr_nom,Pusch_sinr_dnom,Ta_distance_nom,Ta_distance_dnom,Intersite_distance_nom,Intersite_distance_dnom,Rsrp_nom_lag_1_day,Rsrp_dnom_lag_1_day,Rsrq_nom_lag_1_day,Rsrq_dnom_lag_1_day
0,bihar,deoghar,33,2022-11-06,5,4726,Voice,1,4,2,None,27,49,16,10,267,223,62,53,65,223,25,66,9,35,10,31,31,36,9.00,22.00,4.00,0.00,0.00,0,12416679610.00,849541683.00,1393.00,457966.00,16223.00,-852588.00,8767.00,-93934.00,8767.00,172773.00,16154.00,243832.40,15323.00,6892183.00,16223.00,8839869.00,16223.00,-852588.00,8767.00,-93934.00,8767.00
1,bihar,deoghar,33,2022-11-06,6,6551,Voice,1,3,2,None,47,85,27,19,499,495,144,116,160,447,73,124,9,35,10,35,31,33,6.00,19.00,10.00,0.00,0.00,0,22920760951.00,2194595046.00,1808.00,651638.00,20006.00,-1077896.00,11200.00,-127212.00,11200.00,206115.00,19937.00,287435.70,18246.00,7775528.00,20006.00,10909146.00,20006.00,-1077896.00,11200.00,-127212.00,11200.00
2,bihar,katihar,116,2022-11-06,11,13276,HSI,2,2,4,high_prb,137,191,46,48,888,1478,352,375,1533,1532,337,398,8,27,8,32,39,27,0.00,0.00,8.00,8.00,11.00,1,38271955206.00,3384246507.00,2182.00,500347.00,25844.00,-1520924.00,15271.00,-213869.00,15271.00,247706.00,25724.00,118385.60,8936.00,16869064.00,25844.00,44695061.00,25844.00,-1520924.00,15271.00,-213869.00,15271.00
3,bihar,katihar,116,2022-11-06,12,14047,HSI,2,2,4,high_prb,141,197,47,49,851,1454,342,370,1656,1616,351,421,8,27,8,33,40,26,0.00,0.00,7.00,10.00,10.00,1,43646120543.00,3608188983.00,2094.00,480070.00,26834.00,-1679304.00,16734.00,-232907.00,16734.00,252646.00,26701.00,110929.50,8704.00,17197404.00,26834.00,44264376.00,26834.00,-1679304.00,16734.00,-232907.00,16734.00
4,bihar,patna,85,2022-11-06,7,11028,Data - Voice,4,3,0,None,166,129,35,31,991,887,252,211,113,674,131,180,13,60,19,30,31,38,0.00,11.00,41.00,5.00,3.00,0,49107207869.00,4211397132.00,3595.00,1627912.00,50063.00,-2857620.00,29925.00,-379860.50,29925.00,458375.00,49864.00,393226.90,35822.00,24208274.00,50063.00,35345652.00,50063.00,-2857620.00,29925.00,-379860.50,29925.00


In [62]:
anomaly_lsr_df.isnull().sum()

Circle                              0
City                                0
Component                           0
Partition_date                      0
Hour                                0
Total customers                     0
Anomaly                             0
Ratio data                          0
Ratio voice                         0
Ratio hsi                           0
Cluster_issue                   24407
Data volume                         0
Data volume formula                 0
Data volume mean                    0
Data volume std                     0
Voice affected users                0
Voice affected users formula        0
Voice affected users mean           0
Voice affected users std            0
Hsi affected users                  0
Hsi affected users formula          0
Hsi affected users mean             0
Hsi affected users std              0
Num_grids                           0
Num_cells                           0
Num_sectors                         0
Indoor frqeq

In [25]:
## Joining with CCL Data

In [81]:
ccl_df.head()

,Partition_date,Circle,City,Component,Hour,Avg_bad_hsi_pct,Avg_bad_voice_pct
0,2022-11-03,bihar,motihari,151,5,8.05,22.75
1,2022-11-03,bihar,sasaram,1,5,16.77,15.85
2,2022-11-03,bihar,samastipur,72,19,24.92,10.07
3,2022-11-03,bihar,muzaffarpur,54,18,7.68,20.36
4,2022-11-03,bihar,ranchi,122,22,14.59,20.15


In [82]:
ccl_df.shape

(994952, 7)

In [83]:
anomaly_lsr_df.shape

(33310, 56)

In [84]:
ccl_df.dtypes

Partition_date       datetime64[ns]
Circle                       object
City                         object
Component                     Int64
Hour                          Int64
Avg_bad_hsi_pct             float64
Avg_bad_voice_pct           float64
dtype: object

In [85]:
ccl_df['Partition_date']=ccl_df['Partition_date'].astype(str)

In [86]:
anomaly_lsr__ccl_df= pd.merge(anomaly_lsr_df,ccl_df,on=['Circle','City','Partition_date','Component','Hour'],how='left')

In [97]:
anomaly_lsr__ccl_df.head()

,Circle,City,Component,Partition_date,Hour,Total customers,Anomaly,Ratio data,Ratio voice,Ratio hsi,Cluster_issue,Data volume,Data volume formula,Data volume mean,Data volume std,Voice affected users,Voice affected users formula,Voice affected users mean,Voice affected users std,Hsi affected users,Hsi affected users formula,Hsi affected users mean,Hsi affected users std,Num_grids,Num_cells,Num_sectors,Indoor frqeqency % 850,Indoor frqeqency % 1800,Indoor frqeqency % 2300,Prb_10,Prb_10_25,Prb_25_70,Prb_70_90,Prb_90,Target_Cluster,Sum_total_volume_mb_download,Sum_total_volume_mb_upload,Unique_customers,Total_duration_sec,Total_sessions,Rsrp_nom,Rsrp_dnom,Rsrq_nom,Rsrq_dnom,Mean_cqi_nom,Mean_cqi_dnom,Pusch_sinr_nom,Pusch_sinr_dnom,Ta_distance_nom,Ta_distance_dnom,Intersite_distance_nom,Intersite_distance_dnom,Rsrp_nom_lag_1_day,Rsrp_dnom_lag_1_day,Rsrq_nom_lag_1_day,Rsrq_dnom_lag_1_day,Avg_bad_hsi_pct,Avg_bad_voice_pct
0,bihar,deoghar,33,2022-11-06,5,4726,Voice,1,4,2,None,27,49,16,10,267,223,62,53,65,223,25,66,9,35,10,31,31,36,9.00,22.00,4.00,0.00,0.00,0,12416679610.00,849541683.00,1393.00,457966.00,16223.00,-852588.00,8767.00,-93934.00,8767.00,172773.00,16154.00,243832.40,15323.00,6892183.00,16223.00,8839869.00,16223.00,-852588.00,8767.00,-93934.00,8767.00,9.84,19.86
1,bihar,deoghar,33,2022-11-06,6,6551,Voice,1,3,2,None,47,85,27,19,499,495,144,116,160,447,73,124,9,35,10,35,31,33,6.00,19.00,10.00,0.00,0.00,0,22920760951.00,2194595046.00,1808.00,651638.00,20006.00,-1077896.00,11200.00,-127212.00,11200.00,206115.00,19937.00,287435.70,18246.00,7775528.00,20006.00,10909146.00,20006.00,-1077896.00,11200.00,-127212.00,11200.00,9.34,21.39
2,bihar,katihar,116,2022-11-06,11,13276,HSI,2,2,4,high_prb,137,191,46,48,888,1478,352,375,1533,1532,337,398,8,27,8,32,39,27,0.00,0.00,8.00,8.00,11.00,1,38271955206.00,3384246507.00,2182.00,500347.00,25844.00,-1520924.00,15271.00,-213869.00,15271.00,247706.00,25724.00,118385.60,8936.00,16869064.00,25844.00,44695061.00,25844.00,-1520924.00,15271.00,-213869.00,15271.00,0.94,43.54
3,bihar,katihar,116,2022-11-06,12,14047,HSI,2,2,4,high_prb,141,197,47,49,851,1454,342,370,1656,1616,351,421,8,27,8,33,40,26,0.00,0.00,7.00,10.00,10.00,1,43646120543.00,3608188983.00,2094.00,480070.00,26834.00,-1679304.00,16734.00,-232907.00,16734.00,252646.00,26701.00,110929.50,8704.00,17197404.00,26834.00,44264376.00,26834.00,-1679304.00,16734.00,-232907.00,16734.00,7.23,11.55
4,bihar,patna,85,2022-11-06,7,11028,Data - Voice,4,3,0,None,166,129,35,31,991,887,252,211,113,674,131,180,13,60,19,30,31,38,0.00,11.00,41.00,5.00,3.00,0,49107207869.00,4211397132.00,3595.00,1627912.00,50063.00,-2857620.00,29925.00,-379860.50,29925.00,458375.00,49864.00,393226.90,35822.00,24208274.00,50063.00,35345652.00,50063.00,-2857620.00,29925.00,-379860.50,29925.00,10.52,14.38


In [100]:
prb_df.head()

,Partition_date,City,Component,Hour,Ph_mean_prb_dl,Totprbdlmin_perc,Totprbdlmax_perc,Totprbdltot_perc,Totprbdlcnt_cnt
0,2022-11-03,darbhanga,3,19,78.07,29.75,97.83,12541.18,175.80
1,2022-11-03,gaya,15,6,20.54,3.67,63.21,2516.73,175.75
2,2022-11-03,patna,195,5,9.55,2.60,38.91,1236.47,174.19
3,2022-11-03,lakhisarai,24,20,77.74,30.35,96.65,12445.37,175.82
4,2022-11-03,gopalganj,20,11,66.61,19.70,97.13,10485.31,175.79


In [90]:
prb_df['Partition_date'].replace('2023-02-01','2022-11-03',inplace=True)
prb_df['Partition_date'].replace('2023-02-02','2022-11-04',inplace=True)
prb_df['Partition_date'].replace('2023-02-03','2022-11-05',inplace=True)
prb_df['Partition_date'].replace('2023-02-04','2022-11-06',inplace=True)
prb_df['Partition_date'].replace('2023-02-05','2022-11-07',inplace=True)
prb_df['Partition_date'].replace('2023-02-06','2022-11-08',inplace=True)
prb_df['Partition_date'].replace('2023-02-07','2022-11-09',inplace=True)

In [99]:
prb_df.drop(['Prb_20','Prb_70_90','Prb_90'],axis=1,inplace=True)

In [102]:
prb_df['Circle']='bihar'

In [103]:
final_df=pd.merge(anomaly_lsr__ccl_df,prb_df,on=['Circle','City','Partition_date','Component','Hour'],how='left')

In [105]:
final_df.head()

,Circle,City,Component,Partition_date,Hour,Total customers,Anomaly,Ratio data,Ratio voice,Ratio hsi,Cluster_issue,Data volume,Data volume formula,Data volume mean,Data volume std,Voice affected users,Voice affected users formula,Voice affected users mean,Voice affected users std,Hsi affected users,Hsi affected users formula,Hsi affected users mean,Hsi affected users std,Num_grids,Num_cells,Num_sectors,Indoor frqeqency % 850,Indoor frqeqency % 1800,Indoor frqeqency % 2300,Prb_10,Prb_10_25,Prb_25_70,Prb_70_90,Prb_90,Target_Cluster,Sum_total_volume_mb_download,Sum_total_volume_mb_upload,Unique_customers,Total_duration_sec,Total_sessions,Rsrp_nom,Rsrp_dnom,Rsrq_nom,Rsrq_dnom,Mean_cqi_nom,Mean_cqi_dnom,Pusch_sinr_nom,Pusch_sinr_dnom,Ta_distance_nom,Ta_distance_dnom,Intersite_distance_nom,Intersite_distance_dnom,Rsrp_nom_lag_1_day,Rsrp_dnom_lag_1_day,Rsrq_nom_lag_1_day,Rsrq_dnom_lag_1_day,Avg_bad_hsi_pct,Avg_bad_voice_pct,Ph_mean_prb_dl,Totprbdlmin_perc,Totprbdlmax_perc,Totprbdltot_perc,Totprbdlcnt_cnt
0,bihar,deoghar,33,2022-11-06,5,4726,Voice,1,4,2,None,27,49,16,10,267,223,62,53,65,223,25,66,9,35,10,31,31,36,9.00,22.00,4.00,0.00,0.00,0,12416679610.00,849541683.00,1393.00,457966.00,16223.00,-852588.00,8767.00,-93934.00,8767.00,172773.00,16154.00,243832.40,15323.00,6892183.00,16223.00,8839869.00,16223.00,-852588.00,8767.00,-93934.00,8767.00,9.84,19.86,11.67,2.08,49.56,1360.46,173.61
1,bihar,deoghar,33,2022-11-06,6,6551,Voice,1,3,2,None,47,85,27,19,499,495,144,116,160,447,73,124,9,35,10,35,31,33,6.00,19.00,10.00,0.00,0.00,0,22920760951.00,2194595046.00,1808.00,651638.00,20006.00,-1077896.00,11200.00,-127212.00,11200.00,206115.00,19937.00,287435.70,18246.00,7775528.00,20006.00,10909146.00,20006.00,-1077896.00,11200.00,-127212.00,11200.00,9.34,21.39,23.32,2.92,69.31,2859.01,175.75
2,bihar,katihar,116,2022-11-06,11,13276,HSI,2,2,4,high_prb,137,191,46,48,888,1478,352,375,1533,1532,337,398,8,27,8,32,39,27,0.00,0.00,8.00,8.00,11.00,1,38271955206.00,3384246507.00,2182.00,500347.00,25844.00,-1520924.00,15271.00,-213869.00,15271.00,247706.00,25724.00,118385.60,8936.00,16869064.00,25844.00,44695061.00,25844.00,-1520924.00,15271.00,-213869.00,15271.00,0.94,43.54,48.62,11.19,93.67,7589.50,175.89
3,bihar,katihar,116,2022-11-06,12,14047,HSI,2,2,4,high_prb,141,197,47,49,851,1454,342,370,1656,1616,351,421,8,27,8,33,40,26,0.00,0.00,7.00,10.00,10.00,1,43646120543.00,3608188983.00,2094.00,480070.00,26834.00,-1679304.00,16734.00,-232907.00,16734.00,252646.00,26701.00,110929.50,8704.00,17197404.00,26834.00,44264376.00,26834.00,-1679304.00,16734.00,-232907.00,16734.00,7.23,11.55,57.34,11.84,96.62,8736.38,175.75
4,bihar,patna,85,2022-11-06,7,11028,Data - Voice,4,3,0,None,166,129,35,31,991,887,252,211,113,674,131,180,13,60,19,30,31,38,0.00,11.00,41.00,5.00,3.00,0,49107207869.00,4211397132.00,3595.00,1627912.00,50063.00,-2857620.00,29925.00,-379860.50,29925.00,458375.00,49864.00,393226.90,35822.00,24208274.00,50063.00,35345652.00,50063.00,-2857620.00,29925.00,-379860.50,29925.00,10.52,14.38,45.71,9.04,83.81,6210.03,175.75


In [94]:
final_df.isnull().sum()

Circle                              0
City                                0
Component                           0
Partition_date                      0
Hour                                0
Total customers                     0
Anomaly                             0
Ratio data                          0
Ratio voice                         0
Ratio hsi                           0
Cluster_issue                   19161
Data volume                         0
Data volume formula                 0
Data volume mean                    0
Data volume std                     0
Voice affected users                0
Voice affected users formula        0
Voice affected users mean           0
Voice affected users std            0
Hsi affected users                  0
Hsi affected users formula          0
Hsi affected users mean             0
Hsi affected users std              0
Num_grids                           0
Num_cells                           0
Num_sectors                         0
Indoor frqeq

In [106]:
Training_features_path = loc_path + 'Causal Model/05. Training Data/'
final_df.to_csv(Training_features_path +'ADS_Bihar_Nov.csv',index=False)
